In [ ]:
# Load model
from pathlib import Path
import os, glob, torch
from ultralytics import YOLO

def find_best_pt():
    for var in ("improve_dir", "finetune_dir", "run_dir"):
        if var in globals():
            p = Path(globals()[var]) / "weights" / "best.pt"
            if p.exists():
                return str(p)

    cands = glob.glob(str(Path.cwd() / "runs" / "detect" / "*" / "weights" / "best.pt"))
    assert cands, "Tidak menemukan runs/**/weights/best.pt"

    PREFER = ("finetune_improve", "finetune_unfreeze", "finetune")
    def score(p):
        name = Path(p).parents[1].name.lower()
        bonus = max((10*(i+1) for i,tag in enumerate(PREFER) if tag in name), default=0)
        return (bonus, os.path.getmtime(p))
    return max(cands, key=score)

MODEL = find_best_pt()
CONF, IOU, IMG = 0.40, 0.60, 160

print("MODEL:", MODEL, "| conf:", CONF, "iou:", IOU, "imgsz:", IMG)

device = 0 if torch.cuda.is_available() else "cpu"
half = bool(torch.cuda.is_available())

model = YOLO(MODEL)
model.to(device); model.fuse()


MODEL: c:\Users\Ali\Documents\p2-final-project-ftds-030-hck-group01-abulearn-project\Final Project Group 1 HCK 030\p2-final-project-ftds-030-hck-group01-abulearn-project\runs\detect\finetune_unfreeze\weights\best.pt | conf: 0.4 iou: 0.6 imgsz: 160
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


In [ ]:
# Batch folder test_image 
import os

OUT_NAME = "predict_test_image"     # Output folder
res = model.predict(
    source="inf_images",            # Input folder
    imgsz=IMG, conf=CONF, iou=IOU,
    device=device, half=half, verbose=False,
    save=True,                   
    project="runs/detect", name=OUT_NAME, exist_ok=True)

pred_dir = os.path.join("runs", "detect", OUT_NAME)
print("Predictions saved to:", pred_dir)

Results saved to C:\Users\Ali\Documents\p2-final-project-ftds-030-hck-group01-abulearn-project\Final Project Group 1 HCK 030\p2-final-project-ftds-030-hck-group01-abulearn-project\runs\detect\predict_test_image
Predictions saved to: runs\detect\predict_test_image


In [ ]:
# Local webcam inference
import cv2

CONF = 0.55
IOU  = 0.45

for r in model.predict(     
    source=0, stream=True,
    imgsz=IMG, conf=CONF, iou=IOU,
    agnostic_nms=True,       
    device=device, half=half, verbose=False
):
    frame = r.plot()
    cv2.imshow("Webcam - press q to quit", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)

WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING Waiting for stream 0
WARNING W